# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing
from robot.al5d_position_controller import RobotPosition

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
run = "vgg19_128"
exp = Config().get_experiment(experiment, run)

### Check the model for an encoding mode

Check whether we can load back the model

In [2]:
exp = Config().get_experiment(experiment, run)
hidden_size = exp["latent_dims"]
output_size = Config()["robot"]["action_space_size"]

if exp['model'] == 'VGG19ProprioTunedRegression':
    enc = VGG19ProprioTunedRegression(
        hidden_size=hidden_size, output_size=output_size)
else:
    raise Exception(f"Unknown model {exp['model']}")

modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
assert modelfile.exists()
enc.load_state_dict(torch.load(modelfile))

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml


NameError: name 'experiment' is not defined

In [ ]:
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
enc = enc.to(device)

inputlist = []
targetlist = []
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            z = enc.encode(sensor_readings)
            break

print(z)
print(z.shape)


### Show how to use an SensorProcessing object

In [ ]:
# Create the sensor processing package

exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)
sp = VGG19ProprioTunedSensorProcessing(exp, device=device)

In [ ]:
# Run the sensor processing package to process the images of a demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            sensor_readings = sensor_readings.to(device)
            z = sp.process(sensor_readings)
            break

print(z)
print(z.shape)

In [ ]:
# Run the sensor processing inside the BCDemonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
        for i in range(bcd.trim_from, bcd.trim_to):
            #sensor_readings, _ = bcd.get_image(i)
            #sensor_readings = sensor_readings.to(device)
            #z = sp.process(sensor_readings)
            z = bcd.get_z(i)
            break

print(z)
print(z.shape)